# Graphical Lasso Demonstration
This notebook performs the Graphical Lasso on a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the $4$-dimensional graph with two cliques $\{1,2,3\}$ and $\{2 , 3 , 4\}$. The estimation is performed using:

1. **`graphical_lasso()`** from `sklearn.covariance`
2. A **custom implementation** solved using the **BFGS algorithm** from `scipy.optimize.minimize`

## 1. Define the graph and precision matrix

In [7]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  3, 0],
    [ 5, 10,  5,  3],
    [ 3,  5, 10 ,   5],
    [ 0,  3,  5, 10]
])
Sigma = np.linalg.inv(Theta)

## 2. Simulate Gaussian sample

In [32]:
import random 
random.seed(7)
N = 10**4
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[-0.08481423, -0.14148212,  0.50104707, -0.13408804],
       [ 0.01500972, -0.09995599, -0.31503787, -0.05363327],
       [-0.79674774,  0.70858684,  0.41542281, -0.16985076],
       [ 0.13601284, -0.27093846, -0.09018928,  0.27965782],
       [-0.19631773,  0.10019048,  0.36549836, -0.25742948]])

In [33]:
EstimSigma = np.cov(X, rowvar=False)
EstimSigma

array([[ 0.14357317, -0.06813447, -0.02699007,  0.03311031],
       [-0.06813447,  0.16791951, -0.0507427 , -0.02354432],
       [-0.02699007, -0.0507427 ,  0.16574231, -0.06764802],
       [ 0.03311031, -0.02354432, -0.06764802,  0.13810744]])

## 3. Graphical Lasso Using `sklearn.covariance.graphical_lasso`
The `graphical_lasso()` function solves the penalized likelihood problem

$$
argmax_\Theta \; \log\det(\Theta) - \operatorname{tr}(S\Theta) - \lambda \|\Theta\|_1
$$

where:
- **S** is the sample covariance matrix
- **λ** controls the sparsity of the solution
- The algorithm used is the **GLasso algorithm** from Friedman et al. (2008, *Biostatistics*)


In [35]:
from sklearn.covariance import  graphical_lasso

covariance , precision = graphical_lasso(EstimSigma , alpha=  8* pow(10 , -5))

print("Precision matrix (Theta):\n", precision)
print("Covariance matrix (Sigma):\n", covariance)

Precision matrix (Theta):
 [[ 9.89783259  4.95307758  3.12110398 -0.        ]
 [ 4.95307758  9.89543835  5.03547702  2.95954525]
 [ 3.12110398  5.03547702 10.14766418  5.07332757]
 [-0.          2.95954525  5.07332757 10.22565074]]
Covariance matrix (Sigma):
 [[ 0.14357317 -0.06805475 -0.02691088  0.03304815]
 [-0.06805475  0.16791951 -0.05066255 -0.02346429]
 [-0.02691088 -0.05066255  0.16574231 -0.06756802]
 [ 0.03304815 -0.02346429 -0.06756802  0.13810744]]


## 4. Graphical Lasso Using a Custom Optimization Function
We now solve the **same optimization problem** using the **BFGS** algorithm.

In [36]:
from scipy.optimize import minimize
### Helper: Build a Symmetric Matrix from a Vector
def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

### Penalized Log-Likelihood
def penalized_log_likelihood(theta_vec, S , lamb):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    sum_Theta = np.absolute(Theta_mat).sum()    
    return np.log(det) - np.trace(S @ Theta_mat )  - lamb * sum_Theta + lamb * np.absolute(np.diag(Theta_mat)).sum()


### Initial Matrix
A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]


lamb = pow(10 , -4)

### Optimization via BFGS

res = minimize(lambda t: -penalized_log_likelihood(t, EstimSigma , lamb),
               theta0_vec, 
               method = "BFGS"
              )

Theta_MLE = construct_symmetric_matrix(res.x)
Theta_MLE  


array([[ 9.89345975e+00,  4.94841315e+00,  3.11731406e+00,
        -5.66208864e-05],
       [ 4.94841315e+00,  9.88944458e+00,  5.02944184e+00,
         2.95596227e+00],
       [ 3.11731406e+00,  5.02944184e+00,  1.01419062e+01,
         5.06913419e+00],
       [-5.66208864e-05,  2.95596227e+00,  5.06913419e+00,
         1.02218998e+01]])